## Building Tools - LangChain handbook

> https://www.pinecone.io/learn/series/langchain/langchain-tools/

---

### Simple Calculator Tool

We will start with a simple custom tool. The tool is a simple calculator that calculates a circle’s circumference based on the circle’s radius.



In [ ]:
import os
from math import pi, sqrt, cos, sin
from typing import Union, Optional, Type
from dotenv import load_dotenv

In [ ]:
# models and embeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI, HuggingFaceHub
from langchain.docstore.document import Document

from langchain.embeddings import AzureOpenAIEmbeddings

# prompts and utils
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

# loaders and splitters
from langchain_community.document_loaders import DirectoryLoader, TextLoader, UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter, MarkdownTextSplitter, MarkdownHeaderTextSplitter


# memory
from langchain.memory import ConversationBufferWindowMemory

# chains
from langchain.chains import LLMChain, ConversationChain, RetrievalQA

# vectorstores
from langchain.vectorstores import FAISS
from langchain.vectorstores.chroma import Chroma

# agents and tools
from langchain.tools import BaseTool
from langchain.agents import initialize_agent

In [ ]:
load_dotenv(dotenv_path="/Users/shaunaksen/Documents/personal-projects/Natural-Language-Processing/LLM Concepts/llamaindex_tutorials/knowledge_graphs/.env")

In [ ]:
AZURE_API_KEY = os.getenv('AZURE_API_KEY')
AZURE_API_BASE = os.getenv('AZURE_API_BASE')
AZURE_API_VERSION = os.getenv('AZURE_API_VERSION')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

In [ ]:
def init_models() -> list:
    embeddings = AzureOpenAIEmbeddings(
        deployment="text-embedding-ada-002",
        model="text-embedding-ada-002",
        openai_api_type='azure',
        azure_endpoint=AZURE_API_BASE,
        openai_api_key=AZURE_API_KEY,
        openai_api_version=AZURE_API_VERSION,
        chunk_size=1, max_retries=1e0
    )

    llm_chat_gpt_4 = AzureChatOpenAI(deployment_name='gpt-4-32k',
                          model='gpt-4-32k',
                          openai_api_type='azure',
                          azure_endpoint=AZURE_API_BASE,
                          openai_api_key=AZURE_API_KEY,
                          openai_api_version=AZURE_API_VERSION,
                          max_retries=2,
                          temperature=0,
                          streaming=True
                          )

    llm_gpt_4 = AzureOpenAI(deployment_name='gpt-4-32k',
                            model='gpt-4-32k',
                            openai_api_type='azure',
                            azure_endpoint=AZURE_API_BASE,
                            openai_api_key=AZURE_API_KEY,
                            openai_api_version=AZURE_API_VERSION,
                            max_retries=2,
                            temperature=0,
                            streaming=True
                            )

    llm_text_davinci = AzureOpenAI(deployment_name='text-davinci-003',
                          model='text-davinci-003',
                          openai_api_type='azure',
                          azure_endpoint=AZURE_API_BASE,
                          openai_api_key=AZURE_API_KEY,
                          openai_api_version=AZURE_API_VERSION,
                          max_retries=2,
                          temperature=0)

    hub_llm = HuggingFaceHub(
        repo_id="google/flan-t5-large",
        model_kwargs={"temperature": 0}
    )

    return [embeddings, llm_chat_gpt_4, llm_gpt_4, llm_text_davinci, hub_llm]

In [ ]:
embeddings, llm_chat_gpt_4, llm_gpt_4, llm_text_davinci, hub_llm = init_models()

In [ ]:
text = "Tell me an astronaut joke"
print(llm_text_davinci(text))

In [ ]:
class CircumferenceTool(BaseTool):
    name = "Circumference calculator"
    description = "use this tool when you need to calculate a circumference using the radius of a circle"

    def _run(self, radius: Union[int, float]):
        return float(radius)*2.0*pi
    
    def _arun(self, radius: Union[int, float]):
        raise NotImplementedError("This tool does not support async")

Here we initialized our custom CircumferenceTool class using the BaseTool object from LangChain. We can think of the BaseTool as the required template for a LangChain tool.

We have two attributes that LangChain requires to recognize an object as a valid tool. Those are the name and description parameters.

The description is a natural language description of the tool the LLM uses to decide whether it needs to use it. Tool descriptions should be very explicit on what they do, when to use them, and when not to use them.

> In our description, we did not define when not to use the tool. That is because the LLM seemed capable of identifying when this tool is needed. Adding “when not to use it” to the description can help if a tool is overused.

Following this, we have two methods, _run and _arun. When a tool is used, the _run method is called by default. The _arun method is called when a tool is to be used asynchronously. We do not cover async tools in this chapter, so, for now, we initialize it with a NotImplementedError.

In [ ]:
# initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=5,
    return_messages=True
)

Here we initialize the LLM with a temperature of 0. A low temperature is useful when using tools as it decreases the amount of “randomness” or “creativity” in the generated text of the LLMs, which is ideal for encouraging it to follow strict instructions — as required for tool usage.

In the conversation_memory object, we set k=5 to “remember” the previous five human-AI interactions.

Now we initialize the agent itself. It requires the llm and conversational_memory to be already initialized. It also requires a list of tools to be used. We have one tool, but we still place it into a list.


In [ ]:
tools = [CircumferenceTool()]

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm_chat_gpt_4,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

The agent type of chat-conversation-react-description tells us a few things about this agent, those are:

- chat means the LLM being used is a chat model. Both gpt-4 and gpt-3.5-turbo are chat models as they consume conversation history and produce conversational responses. A model like text-davinci-003 is not a chat model as it is not designed to be used this way.

- conversational means we will be including conversation_memory.

- react refers to the ReAct framework, which enables multi-step reasoning and tool usage by giving the model the ability to “converse with itself”.

- description tells us that the LLM/agent will decide which tool to use based on their descriptions — which we created in the earlier tool definition.


In [ ]:
agent("can you calculate the circumference of a circle that has a radius of 7.81mm")

In [ ]:
7.81 * 2 * pi

> This works here (maybe because we are using gpt-4) but in the article this did not work, so for the next step we will assume that the ans returned is not perfect

The Final Answer action is what the agent uses when it has decided it has completed its reasoning and action steps and has all the information it needs to answer the user’s query. That means the agent decided not to use the circumference calculator tool.

LLMs are generally bad at math, but that doesn’t stop them from trying to do math. The problem is due to the LLM’s overconfidence in its mathematical ability. To fix this, we must tell the model that it cannot do math. First, let’s see the current prompt being used:



In [ ]:
type(agent)

In [ ]:
type(agent.agent)

In [ ]:
print (agent.agent.llm_chain.prompt.messages[0])

In [ ]:
print (agent.agent.llm_chain.prompt.messages[0].prompt.template)

We will add a single sentence that tells the model that it is “terrible at math” and should never attempt to do it.

```
Unfortunately, the Assistant is terrible at maths. When provided with math questions, no matter how simple, assistant always refers to its trusty tools and absolutely does NOT try to answer math questions by itself
```

With this added to the original prompt text, we create a new prompt using agent.agent.create_prompt — this will create the correct prompt structure for our agent, including tool descriptions. Then, we update agent.agent.llm_chain.prompt.

In [ ]:
sys_msg = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Unfortunately, Assistant is terrible at maths. When provided with math questions, no matter how simple, assistant always refers to it's trusty tools and absolutely does NOT try to answer math questions by itself

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.
"""

In [ ]:
new_prompt = agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)

In [ ]:
type(new_prompt)

In [ ]:
agent.agent.llm_chain.prompt = new_prompt

In [ ]:
agent("can you calculate the circumference of a circle that has a radius of 7.81mm")

### Tools With Multiple Parameters

In the circumference calculator, we could only input a single value — the radius — more often than not, we will need multiple parameters.


To demonstrate how to do this, we will build a Hypotenuse calculator. The tool will help us calculate the hypotenuse of a triangle given a combination of triangle side lengths and/or angles.


We want multiple inputs here because we calculate the triangle hypotenuse with different values (the sides and angle). Additionally, we don’t need all values. We can calculate the hypotenuse with any combination of two or more parameters.

We define our new tool like so:



In [ ]:
desc = (
    "use this tool when you need to calculate the length of a hypotenuse"
    "given one or two sides of a triangle and/or an angle (in degrees). "
    "To use the tool, you must provide at least two of the following parameters "
    "['adjacent_side', 'opposite_side', 'angle']."
)


In [ ]:
class PythagorasTool(BaseTool):
    name = "Hypotenuse calculator"
    description = desc

    def _run(
            self, 
            adjacent_side: Optional[Union[int, float]]=None,
            opposite_side: Optional[Union[int, float]]=None,
            angle: Optional[Union[int, float]]=None,
    ):
        print (adjacent_side, opposite_side, angle)
        # check for the values we have been given
        if adjacent_side and opposite_side:
            return sqrt(float(adjacent_side)**2 + float(opposite_side)**2)
        elif adjacent_side and angle:
            return adjacent_side / cos(float(angle))
        elif opposite_side and angle:
            return opposite_side / sin(float(angle))
        else:
            return "Could not calculate the hypotenuse of the triangle. Need two or more of `adjacent_side`, `opposite_side`, or `angle`."
        
    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

In [ ]:
PythagorasTool._run(10, 20)

In [ ]:
tools = [PythagorasTool()]

In the tool description, we describe the tool functionality in natural language and specify that to “use the tool, you must provide at least two of the following parameters [‘adjacent_side’, ‘opposite_side’, ‘angle’]". This instruction is all we need for gpt-3.5-turbo to understand the required input format for the function.

As before, we must update the agent’s prompt. We don’t need to modify the system message as we did before, but we do need to update the available tools described in the prompt.



In [ ]:
new_prompt = agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)

agent.agent.llm_chain.prompt = new_prompt

In [ ]:
# must also update the agent.tools attribute with our new tools
agent.tools = tools

In [ ]:
agent("If I have a triangle with adjacent side of length 51cm and opposite side of length 34cm, what is the length of the hypotenuse?")

In [ ]:
agent("If I have a triangle with the opposite side of length 51cm and an angle of 20 deg, what is the length of the hypotenuse?")

> Does not seem to work


Let us go through langchain offcial guide to learn more and see if we can correct this down the line

## Defining Custom Tools

> https://python.langchain.com/docs/modules/agents/tools/custom_tools

---

When constructing your own agent, you will need to provide it with a list of Tools that it can use. Besides the actual function that is called, the Tool consists of several components:

- name (str), is required and must be unique within a set of tools provided to an agent

- description (str), is optional but recommended, as it is used by an agent to determine tool use

- args_schema (Pydantic BaseModel), is optional but recommended, can be used to provide more information (e.g., few-shot examples) or validation for expected parameters.


There are multiple ways to define a tool. In this guide, we will walk through how to do for two functions:

1. A made up search function that always returns the string “LangChain”

2. A multiplier function that will multiply two numbers by eachother


The biggest difference here is that the first function only requires one input, while the second one requires multiple. Many agents only work with functions that require single inputs, so it’s important to know how to work with those. For the most part, defining these custom tools is the same, but there are some differences.

In [ ]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

This `@tool` decorator is the simplest way to define a custom tool. The decorator uses the function name as the tool name by default, but this can be overridden by passing a string as the first argument. Additionally, the decorator will use the function’s docstring as the tool’s description - so a docstring MUST be provided.



In [ ]:
@tool("search")
def search(query: str) -> str:
    """
    Look up things online.
    """
    return "Mini"

In [ ]:
print(search.name)
print(search.description)
print(search.args)

In [ ]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [ ]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

You can also customize the tool name and JSON args by passing them into the tool decorator.

In [ ]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query", default="what is meaning of life?")

@tool("search-tool", args_schema=SearchInput, return_direct=True)
def search(query: str) -> str:
    """
    Look up things online.
    """
    return "Mini"

In [ ]:
print(search.name)
print(search.description)
print(search.args)
print(search.return_direct)

### Subclass BaseTool

You can also explicitly define a custom tool by subclassing the BaseTool class. This provides maximal control over the tool definition, but is a bit more work.

In [ ]:
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun

1. Lets define the schema classes:

In [ ]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

class CalculatorInput(BaseModel):
    a: Union[int, str] = Field(description="first number")
    b: Union[int, str] = Field(description="second number")

2. Define the Custom search tool

In [ ]:
class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer questions about current events"
    args_schema: Type[BaseModel] = SearchInput

    def _run(
            self, query: str, run_manager: Optional[CallbackManagerForToolRun]=None
    ):
        """Use the tool."""
        return "Mini"
    
    async def _arun(
            self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun]=None
    ):
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [ ]:
search = CustomSearchTool()
print(search.name)
print(search.description)
print(search.args)

In [ ]:
class CustomCalculatorTool(BaseTool):
    name = "Calculator"
    description = "useful for when you need to answer questions about math"
    args_schema: Type[BaseModel] = CalculatorInput
    return_direct: bool=True

    def _run(
            self, a: Union[int, str], b: Union[int, str], run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """
        Use the tool
        """
        a = int(a)
        b= int(b)
        return a*b
    
    def _arun(
            self, a: int, b: int, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("Calculator does not support async")


In [ ]:
multiply = CustomCalculatorTool()
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.return_direct)

In [ ]:
import langchain

langchain.__version__

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent

In [ ]:
tools = [CustomCalculatorTool()]

In [ ]:
template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

In [ ]:
prompt = PromptTemplate(
    input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'],
    template=template
)

In [ ]:
agent = create_react_agent(llm_chat_gpt_4, tools, prompt)

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

In [ ]:
agent_executor.invoke({"input": "what is 7 times 7?"})

In [ ]:
tools = [CustomCalculatorTool()]

# initialize agent with tools
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm_chat_gpt_4,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

In [ ]:
class CalculatorInput(BaseModel):
    a: list = Field(description="first number")
    b: Union[int, str] = Field(description="second number")


def multiply(a: Union[int, str], b: Union[int, str]) -> int:
    """Multiply two numbers."""
    return int(a) * int(b)


calculator = StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="multiply numbers",
    args_schema=CalculatorInput,
    return_direct=True,
    # coroutine= ... <- you can specify an async method if desired as well
)

In [ ]:
tools = [calculator]

In [ ]:
agent = create_react_agent(llm_chat_gpt_4, tools, prompt)

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

In [ ]:
agent_executor.invoke({"input": "what is 7 times 7?"})